# Perform a template fit of the for Data = a*QCD + b*VBF
## Using HistFactory in Root

In [106]:
import ROOT
import os
import numpy as np

SF_sherpa = np.array([1.4229031464142285, 1.2636541605686187, 1.1212033450455618, 0.9954504258520799, 0.886329633401572, 0.7938348134176859, 0.6864996611752437, 0.6038065940904263, 0.5960159252170011, 0.7537451120259034, 1.3714787994103248])
SF_sherpa_error = np.array([0.00255163, 0.00154236, 0.00101951, 0.00101349, 0.00120375, 0.00140845, 0.00192416, 0.00367772, 0.00684859, 0.0140738 , 0.02812596])
SF_mg =  np.array([0.9244978960254908, 0.8031821756195101, 0.7008139835646783, 0.6169369856295047, 0.5509350432451292, 0.5020431346726935, 0.4588691005346224, 0.4531065803969615, 0.5023618315269036, 0.6806801799700004, 1.1411394524872462])
SF_mg_error = np.array([0.00255163, 0.00154236, 0.00101951, 0.00101349, 0.00120375, 0.00140845, 0.00192416, 0.00367772, 0.00684859, 0.0140738 , 0.02812596])

def fillHistogramErrors(histogram,binErrors):
    for i in range(1,histogram.GetNbinsX()+1):
        histogram.SetBinError(i,binErrors[i-1])
        
def unpackHisto(histogram_file):
    bin_content_uncer =[]
    for i in range(1,histogram_file.GetNbinsX()+1):
        bin_content_uncer.append(histogram_file.GetBinError(i))
    return np.array(bin_content_uncer)
        
        
inputPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/MuMu/Plots/AnalysisCrossCheck/SR/"

otherBGs = ['Wjets.root','VV.root','ttbar.root','singletop.root']
if "Tau" in inputPath:
    otherBGs += ['Higgs.root','Zjets.root']
samples = {"Signal":"Signal_Sherpa.root","QCDjj":"Zmumu_SherpaRW.root","Data":"Data.root"}
qcd_shape_uncer = []

inputFile = ROOT.TFile.Open("histograms.root","RECREATE")
for i in samples:
    file = ROOT.TFile.Open(inputPath+samples[i])
    histogram = file.Get("mass_jj")
    x = np.array([   0.0,  250.0,  500.0,  750.0, 1000.0, 1250.0, 1500.0, 2000.0, 2500.0, 3000.0, 4000.0, 5000.0])
    
    if "Average" not in samples[i]:
        histogram = histogram.Rebin(len(x)-1,i,x)
    
    if i=="QCDjj":
        qcd_shape_uncer = unpackHisto(histogram)
        print('QCD Integral = ', histogram.Integral(1,-1))
        if "Sherpa" in samples["QCDjj"]:
            SF = SF_sherpa
            SF_error = SF_sherpa_error 
        elif "MG" in samples["QCDjj"]:
            SF = SF_mg
            SF_error = SF_mg_error 
        if "Average" not in samples[i]:
            pass
            #qcd_shape_uncer = np.sqrt(qcd_shape_uncer**2+((SF_error/SF)**2)*qcd_shape_uncer**2) 
            #fillHistogramErrors(histogram,qcd_shape_uncer)
        
    if i == "Data":
        for j in otherBGs:
            BGFile = ROOT.TFile.Open(inputPath+j)
            BGHistogram = BGFile.Get("mass_jj")
            BGHistogram = BGHistogram.Rebin(len(x)-1,j,x)
            
            histogram.Add(BGHistogram,-1.0)
            
    inputFile.WriteObject(histogram,i)
    file.Close()
    
inputFile.Close()

QCD Integral =  13229.93245601654


In [107]:
os.system("root HistFactoryFit.cpp | tail -n 2")

0

VBF = 1.17278 +/- 0.0565777
QCD = 0.940965 +/- 0.011342


Info in <TCanvas::Print>: eps file ./results/example_UsingPy_channel1_meas_profileLR.eps has been created
Info in <TCanvas::Print>: eps file ./results/example_UsingPy_combined_meas_profileLR.eps has been created


## Using plain iminuit

In [26]:
import iminuit
import matplotlib.pyplot as plt
import numpy as np

from iminuit import cost, Minuit

def createHistogram(histogram):
    histo = []
    for i in histogram:
        histo.append([i,np.sqrt(i)])
    return np.array(histo)

yDataMeasurements = np.random.normal(91,4,1000)
yPredictionMeasurements = np.random.normal(91,4,100)

np.random.exponential()

data , x = np.histogram(yDataMeasurements,bins=200,range=(0,200))
prediction, x = np.histogram(yPredictionMeasurements,bins=200,range=(0,200))


dataHistogram = createHistogram(data)
predictionHistogram = createHistogram(prediction)

c = cost.Template(dataHistogram, x, np.array([predictionHistogram]))
m = Minuit(c,1000.0)
m.migrad()